In [11]:
import numpy as np
import pandas as pd

In [20]:
! pip install opencv-python

In [19]:
import cv2 

In [275]:
clean_labels == noisy_labels.iloc[:1000,]

,0
0,3
1,5
2,9
3,6
4,5
...,...
995,2
996,3
997,0
998,5


In [140]:
clean_labels = pd.read_csv("train_data/clean_labels.csv", header = None)
noisy_labels = pd.read_csv("train_data/noisy_labels.csv", header = None)

In [13]:
print(clean_labels.value_counts())

2    1032
6    1030
8    1025
3    1016
0    1005
7    1001
4     999
9     981
1     974
5     937
dtype: int64


In [20]:
n_img = 50000
all_imgs = np.empty((n_img, 32, 32, 3))

for i in range(n_img):
    img_fn = f'train_data/images/{i+1:05d}.png'
    all_imgs[i,:,:,:]=cv2.cvtColor(cv2.imread(img_fn),cv2.COLOR_BGR2RGB)

In [21]:
clean_imgs = all_imgs[:10000,:,:,:]
noisy_imgs = all_imgs[10000:,:,:,:]

what each label is:

1-car
2-bird
3-cat
4-deer
5-dog
6-frog
7-horse
8-ship
9-truck

In [169]:
import keras
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Concatenate, Add, Input, Embedding
from keras.layers import Conv2D, MaxPooling2D, LeakyReLU
from tensorflow.keras.utils import to_categorical

In [106]:
# clean_imgs_shaped = np.reshape(clean_imgs,(10000,-1))
# noisy_imgs_shaped = np.reshape(noisy_imgs,(40000,-1))
# clean_imgs_shaped.shape

(10000, 3072)

In [178]:
# one hot label data
clean_labs = to_categorical(clean_labels,num_classes = 10)
noisy_labs = to_categorical(noisy_labels,num_classes = 10)

In [310]:
# cleaning model construction
input_labels = Input(shape = (10,))
xl = Dense(64)(input_labels)

input_images = Input(shape = (32,32,3,))
xi = Conv2D(32,(3,3),1,padding="same",activation = "linear")(input_images)
xi = LeakyReLU(0.1)(xi)
xi = MaxPooling2D((2,2),padding="same")(xi)
xi = Dropout(0.2)(xi)
xi = Conv2D(64,(3,3),1,padding="same",activation = "linear")(xi)
xi = LeakyReLU(0.1)(xi)
xi = MaxPooling2D((2,2),padding="same")(xi)
xi = Dropout(0.2)(xi)
xi = Flatten()(xi)
xi = Dense(64, activation = "linear")(xi)

x = Concatenate()([xl,xi])
x = Dense(32)(x)
x = Dropout(0.2)(x)
x = Dense(10, activation = "softmax")(x)
y = Add()([input_labels,x]) #link connection to mitigate overfitting

In [311]:
model = Model(inputs = [input_labels,input_images], outputs = y, name = "label_cleaner")

In [312]:
model.compile(loss = "categorical_crossentropy", metrics = ["accuracy"], optimizer = "Adam")

In [301]:
model.summary()

Model: "label_cleaner"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_68 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_53 (Conv2D)             (None, 32, 32, 32)   896         ['input_68[0][0]']               
                                                                                                  
 leaky_re_lu_19 (LeakyReLU)     (None, 32, 32, 32)   0           ['conv2d_53[0][0]']              
                                                                                                  
 max_pooling2d_50 (MaxPooling2D  (None, 16, 16, 32)  0           ['leaky_re_lu_19[0][0]']         
 )                                                                                    

How the data splits work: labels -> {0,1} is right label or wrong label
Have: 50000 images I, 10000 clean data C, 40000 noisy data w/o clean NO, 10000 clean data w/o clean NC
Generate from 1: 40000 cleaned data of noisy GN, 10000 cleaned data of clean data GC

Train 1: 1 NO - 0.8 I 
Valid 1: 0.6 C, 0.6 NC - 0.6*0.2 I
Test 1: 0.4 C, 0.4 NC - 0.4*0.2 I

---- unimportant for now
Train 2: 1 I, 
Valid 2: 
Test 2: (draw test from train2)

In [313]:
#7:2:1 split
train_split = np.random.choice(1000,size = 700,replace = False)
valid_split = np.random.choice(np.delete(np.arange(1000),train_split),size = 200,replace = False)
test_split = np.delete(np.arange(1000),np.append(valid_split,train_split))

In [314]:
train1_imgs = clean_imgs[train_split,]/255.0
valid1_imgs = clean_imgs[valid_split,]/255.0
test1_imgs = clean_imgs[test_split,]/255.0

train1_labels = noisy_labs[train_split,]
ytrain1_labels = clean_labs[train_split,]
valid1_labels = noisy_labs[valid_split,]
yvalid1_labels = clean_labs[valid_split,]
test1_labels = noisy_labs[test_split,]
ytest1_labels = clean_labs[test_split,]

In [214]:
import time

In [315]:
start = time.time()
cleaner_train = model.fit([train1_labels,train1_imgs], ytrain1_labels, epochs = 20, batch_size = 128, verbose=1, validation_data = ([valid1_labels,valid1_imgs],yvalid1_labels))
end = time.time()
print("Training took " + str((end-start)/60) + " minutes")

Epoch 1/20
6/6 [==============================] - 3s 225ms/step - loss: 2.1442 - accuracy: 0.3871 - val_loss: 2.1753 - val_accuracy: 0.3550
Epoch 2/20
6/6 [==============================] - 1s 153ms/step - loss: 2.0646 - accuracy: 0.3871 - val_loss: 2.1394 - val_accuracy: 0.3550
Epoch 3/20
6/6 [==============================] - 1s 163ms/step - loss: 2.0319 - accuracy: 0.3871 - val_loss: 2.1097 - val_accuracy: 0.3550
Epoch 4/20
6/6 [==============================] - 1s 157ms/step - loss: 1.9720 - accuracy: 0.3871 - val_loss: 2.0568 - val_accuracy: 0.3550
Epoch 5/20
6/6 [==============================] - 1s 154ms/step - loss: 1.9223 - accuracy: 0.3871 - val_loss: 2.0013 - val_accuracy: 0.3550
Epoch 6/20
6/6 [==============================] - 1s 174ms/step - loss: 1.8698 - accuracy: 0.3871 - val_loss: 1.9649 - val_accuracy: 0.3550
Epoch 7/20
6/6 [==============================] - 1s 199ms/step - loss: 1.7990 - accuracy: 0.3871 - val_loss: 1.9193 - val_accuracy: 0.3550
Epoch 8/20
6/6 [====

In [309]:
# test_loss, test_accuracy = model.evaluate([test1_labels,test1_imgs], ytest1_labels)

4/4 [==============================] - 0s 11ms/step - loss: 1.5677 - accuracy: 0.4200


In [297]:
model.predict([valid1_labels,valid1_imgs])[0]

7/7 [==============================] - 0s 12ms/step


array([4.0055133e-02, 1.0533844e-02, 3.4285724e-04, 5.8615278e-04,
       1.0029292e+00, 3.4661102e-04, 1.4382631e-04, 4.2386305e-01,
       1.0116595e-02, 5.1108265e-01], dtype=float32)